## **Elaborazione di Immagini Mediche**
### 2021/22 - VESSELWALL SEGMENTATION CHALLENGE

# Import

In [1]:
import os
import pydicom
import glob
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import shutil

import xml.etree.ElementTree as ET

import scipy.ndimage as ndimage
from scipy.spatial.distance import pdist,squareform
from scipy.interpolate import interp1d

import skimage
from skimage.io import imread, imshow, imsave
from skimage.draw import polygon
from copy import deepcopy
import re

# Functions

In [2]:
def readDicom(path):
    pi = os.path.basename(path).split('_')[1]
    dcm_size = len(glob.glob(path+'/*.dcm'))
    dcms = [path+'/E'+pi+'S101I%d.dcm'%dicom_slicei for dicom_slicei in range(1,dcm_size+1)]
            
    dcm_f = pydicom.read_file(dcms[0]).pixel_array
    dcm_size = max(dcm_f.shape)

    cdcm1 = pydicom.read_file(dcms[0]).pixel_array

    cdcm_img = np.zeros((cdcm1.shape[0],cdcm1.shape[1],len(dcms)))
    dcm_img = np.zeros((dcm_size,dcm_size,len(dcms)))
    for dcmi in range(len(dcms)):
        if os.path.exists(dcms[dcmi])==True:
            cdcm = pydicom.read_file(dcms[dcmi]).pixel_array
            dcm_img[dcm_size//2-cdcm.shape[0]//2:dcm_size//2+cdcm.shape[0]//2,
                    dcm_size//2-cdcm.shape[1]//2:dcm_size//2+cdcm.shape[1]//2,dcmi] = cdcm
            cdcm_img[:,:,dcmi] = cdcm

    return dcm_img, cdcm_img 

def listContourSlices(qvsroot):
    avail_slices = []
    qvasimg = qvsroot.findall('QVAS_Image')
    for dicom_slicei in range(dcm_img.shape[2]):
        conts = qvasimg[dicom_slicei - 1].findall('QVAS_Contour')
        if len(conts):
            avail_slices.append(dicom_slicei)
    return avail_slices

def getContour(qvsroot,dicomslicei,conttype,dcmsz=720):
    qvasimg = qvsroot.findall('QVAS_Image')
    if dicomslicei - 1 > len(qvasimg):
        print('no slice', dicomslicei)
        return
    assert int(qvasimg[dicomslicei - 1].get('ImageName').split('I')[-1]) == dicomslicei
    conts = qvasimg[dicomslicei - 1].findall('QVAS_Contour')
    tconti = -1
    for conti in range(len(conts)):
        if conts[conti].find('ContourType').text == conttype:
            tconti = conti
            break
    if tconti == -1:
        print('no such contour', conttype)
        return
    pts = conts[tconti].find('Contour_Point').findall('Point')
    contours = []
    for pti in pts:
        contx = float(pti.get('x')) / 512 * dcmsz 
        conty = float(pti.get('y')) / 512 * dcmsz 
        #if current pt is different from last pt, add to contours
        if len(contours) == 0 or contours[-1][0] != contx or contours[-1][1] != conty:
            contours.append([contx, conty])
    return np.array(contours)

In [3]:
def sorted_alphanum(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

# Preliminary Dataset

In [4]:
data_type = ('DATASET','DATASET_Intermedio')
set_type = ('TRAIN','VALIDATION')
masks_name = ('ICAL','ECAL','ICAR','ECAR')
result_str = 'RESULTS'

IMG_WIDTH = 720
IMG_HEIGHT = 100

path = os.getcwd()

* Unzip Dataset iniziale

In [5]:
if not os.path.exists(os.path.join(path,data_type[0])):
    # !unzip os.path.join(path,data_type[0]+'.zip')
    shutil.unpack_archive(data_type[0]+'.zip', path, format='zip')

* Crea cartelle

In [6]:
train_dir = os.path.join(path,data_type[0],set_type[0])
val_dir = os.path.join(path,data_type[0],set_type[1])

newTrain_dir = os.path.join(path,data_type[1],set_type[0])
newVal_dir = os.path.join(path,data_type[1],set_type[1])

# Create 'Results folder'
if not os.path.exists(os.path.join(path,result_str)):
    os.mkdir(os.path.join(path,result_str))

# Create 'New Data folder'
if not os.path.exists(os.path.join(path,data_type[1])):
    os.mkdir(os.path.join(path,data_type[1]))

# Create 'Set folders in New Data Folder'
for sett in set_type:
    if not os.path.exists(os.path.join(path,data_type[1],sett)):
        os.mkdir(os.path.join(path,data_type[1],sett))

listOfPz_train = sorted_alphanum(os.listdir(train_dir))     
listOfPz_val = sorted_alphanum(os.listdir(val_dir))     #Change to change Set where create masks

## Estrazione slice e maschere

In [ ]:
# Cambiare le 3 righe sottostanti per processare train o validation
listOfPz = listOfPz_val          #listOfPz_train
dir_ToRead = val_dir          #train_dir 
dir_ToSave = newVal_dir         #newTrain_dir

dict_stamp = {  'ICAL':[],
                'ECAL':[],
                'ICAR':[],
                'ECAR':[]
                }

for pz in listOfPz:
    diameters = deepcopy(dict_stamp)
    slicesNoContour = deepcopy(dict_stamp)
    slicesOutImg = deepcopy(dict_stamp)
    slicesBiforc = deepcopy(dict_stamp)
    avail_slices_final = deepcopy(dict_stamp)
    avail_slices_new = deepcopy(dict_stamp)

    flag_biforc = False

    # Crea cartella per paziente
    if len(listOfPz) == len(listOfPz_train) and not os.path.exists(os.path.join(newTrain_dir,pz)):
        os.mkdir(os.path.join(newTrain_dir,pz))

    if len(listOfPz) == len(listOfPz_val) and not os.path.exists(os.path.join(newVal_dir,pz)):
        os.mkdir(os.path.join(newVal_dir,pz))

    pi = pz.split('_')[1]
    dcm_img, cdcm_img = readDicom(dir_ToRead+'/'+pz)
    
    for mask_name in masks_name:
        mask_dir = dir_ToRead+'/'+pz+'/CASCADE-'+mask_name
        qvs_path = mask_dir+'/E'+pi+'S101_L.QVS'
        if os.path.exists(mask_dir)==True:
            qvsroot = ET.parse(qvs_path).getroot()
            avail_slices = listContourSlices(qvsroot)
            avail_slices_backup = avail_slices
            qvasimg = qvsroot.findall('QVAS_Image')
            
            if len(avail_slices):
                nRows_cdcm = cdcm_img.shape[0]
                nRows_dcm = dcm_img.shape[0]
                nCols_cdcm = cdcm_img.shape[1]
                nCols_dcm = dcm_img.shape[1]
                cont = 0
                for avail_slice in avail_slices:
                    lumen_cont = getContour(qvsroot,avail_slice,'Lumen',nCols_dcm)
                    wall_cont = getContour(qvsroot,avail_slice,'Outer Wall',nCols_dcm)

                    try:
                        lumen_cont_cdcm = np.zeros_like(lumen_cont)
                        wall_cont_cdcm = np.zeros_like(wall_cont)

                        lumen_cont_cdcm[:,0] = lumen_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                        lumen_cont_cdcm[:,1] = lumen_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)             
                        
                        wall_cont_cdcm[:,0] = wall_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                        wall_cont_cdcm[:,1] = wall_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)

                        dist = pdist(lumen_cont_cdcm)     # calcolo distanza tra tutti i pixel del contorno
                        dist = squareform(dist)         
                        max_dist = np.max(dist)          # calcolo diametro (distanza massima)
                        idx_max_dist = np.argmax(max_dist)       # calcolo indice diametro
                        
                        diameters[mask_name].append([avail_slice,max_dist])
                    except:
                        pass   
                
                for rows in diameters[mask_name]:
                    avail_slices_new[mask_name].append(rows[0])

    for mask_name in masks_name:
        mask_dir = dir_ToRead+'/'+pz+'/CASCADE-'+mask_name
        qvs_path = mask_dir+'/E'+pi+'S101_L.QVS'
        if os.path.exists(mask_dir)==True:
            qvsroot = ET.parse(qvs_path).getroot()
                 
            if len(avail_slices_new[mask_name]):           
                max_diam = 0
                for current in diameters[mask_name]:
                    if current[1] > max_diam:                  # trovo diametro massimo all'interno del volume
                        max_diam = current[1]
                        slice_max_diam = current[0]

                # individuo area biforcazione e slices da eliminare:
                AreaBiforc = 29   # informazione ricavata dalla consegna
                if mask_name=='ICAL' or mask_name=='ICAR': 
                    idx_slice_startBiforc = avail_slices_new[mask_name].index(slice_max_diam)
                    firstSliceToRemove = avail_slices_new[mask_name][idx_slice_startBiforc-3]
                    lastSliceToRemove = avail_slices_new[mask_name][idx_slice_startBiforc] + AreaBiforc
                    flag_biforc = True

                if avail_slices_new['ICAL'] == [] or avail_slices_new['ICAR'] == []:
                    if mask_name=='ECAL' or mask_name=='ECAR': 
                        idx_slice_startBiforc = avail_slices_new[mask_name].index(slice_max_diam)
                        firstSliceToRemove = avail_slices_new[mask_name][idx_slice_startBiforc-3]
                        lastSliceToRemove = avail_slices_new[mask_name][idx_slice_startBiforc] + AreaBiforc
                        flag_biforc = True
                
                if flag_biforc==True:
                    for avail_slice in avail_slices_new[mask_name]:
                        if firstSliceToRemove < avail_slice < lastSliceToRemove:
                            slicesBiforc[mask_name].append(avail_slice)                  
                      
                slices_to_remove = slicesBiforc[mask_name]
                for slice_to_remove in slices_to_remove:
                    avail_slices_new[mask_name].remove(slice_to_remove)              
                
                avail_slices_final[mask_name] = avail_slices_new[mask_name]
                
    for mask_name in masks_name:
        mask_dir = dir_ToRead+'/'+pz+'/CASCADE-'+mask_name
        qvs_path = mask_dir+'/E'+pi+'S101_L.QVS'
        if os.path.exists(mask_dir)==True:
            qvsroot = ET.parse(qvs_path).getroot()
            if len(avail_slices_final[mask_name]):
                nRows_cdcm = cdcm_img.shape[0]
                nRows_dcm = dcm_img.shape[0]
                nCols_cdcm = cdcm_img.shape[1]
                nCols_dcm = dcm_img.shape[1]
                for avail_slice in avail_slices_final[mask_name]:
                    lumen_cont = getContour(qvsroot,avail_slice,'Lumen',nCols_dcm)
                    wall_cont = getContour(qvsroot,avail_slice,'Outer Wall',nCols_dcm)

                    lumen_cont_cdcm = np.zeros_like(lumen_cont)
                    wall_cont_cdcm = np.zeros_like(wall_cont)

                    lumen_cont_cdcm[:,0] = lumen_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                    lumen_cont_cdcm[:,1] = lumen_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)              
                    
                    wall_cont_cdcm[:,0] = wall_cont[:,0] - (nCols_dcm/2-nCols_cdcm/2)
                    wall_cont_cdcm[:,1] = wall_cont[:,1] - (nRows_dcm/2-nRows_cdcm/2)

                    mask_lumen = np.zeros_like(cdcm_img[:,:,1], dtype='int')
                    mask_wall = np.zeros_like(cdcm_img[:,:,1], dtype='int')
                    try:
                        # ridisegno le maschere a partire dai contorni
                        r_l, c_l = polygon(lumen_cont_cdcm[:,0].astype(int), lumen_cont_cdcm[:,1].astype(int))
                        r_w, c_w = polygon(wall_cont_cdcm[:,0].astype(int), wall_cont_cdcm[:,1].astype(int))

                        mask_lumen[np.round(c_l), np.round(r_l)] = 1
                        mask_wall[np.round(c_w), np.round(r_w)] = 1

                        mask_lumen = ndimage.binary_fill_holes(mask_lumen)
                        mask_wall = ndimage.binary_fill_holes(mask_wall)

                        mask_wall = np.logical_xor(mask_wall,mask_lumen)
                        
                        img_final = cdcm_img[:,:,avail_slice]
                        
                        if img_final.shape == (IMG_HEIGHT,IMG_WIDTH):    # salvo solo slice di dimensioni 100x720
                            if avail_slice >= 400:     #assegno la label 'ICA' a tutte le slice riguardanti la corotide comune
                                if mask_name == 'ECAL':
                                    mask_name = 'ICAL'
                                elif mask_name == 'ECAR':
                                    mask_name = 'ICAR'
                            imsave(os.path.join(dir_ToSave,pz,'E'+pi+'S101I'+str(avail_slice)+'.png'),img_final)
                            imsave(os.path.join(dir_ToSave,pz,'E'+pi+'S101I'+str(avail_slice)+'_'+mask_name+'_'+'lume'+'.png'),mask_lumen)
                            imsave(os.path.join(dir_ToSave,pz,'E'+pi+'S101I'+str(avail_slice)+'_'+mask_name+'_'+'wall'+'.png'),mask_wall)
                    except:
                        slicesOutImg[mask_name].append(avail_slice)